# Delaunay-Watershed for mesh creation: cell aggregate example

In this notebook, we present how to compress segmentation masks of cells in an aggregate into a multimaterial mesh.
We show that one can recompute the mask images from the mesh and seeds for segmentation algorithms if necessary.

## 1) Compute multimaterial mesh with Delaunay Watershed

In [ ]:
pip install -r ../addition_requirements.txt

  Obtaining dependency information for csbdeep from https://files.pythonhosted.org/packages/e7/19/d00c5b9ba50b825b183a3242be1da8f9e2685386bbea5ee679167996cff5/csbdeep-0.7.4-py2.py3-none-any.whl.metadata
  Using cached csbdeep-0.7.4-py2.py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for napari[all] from https://files.pythonhosted.org/packages/4e/08/3e0abcb077b5633ee029f601b9499e9fa7ca252fc9656a6957d993f60cfd/napari-0.4.18-py3-none-any.whl.metadata
  Using cached napari-0.4.18-py3-none-any.whl.metadata (14 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Obtaining dependency information for app-model<0.3.0,>=0.1.0 from https://files.pythonhosted.org/packages/e2/30/153aa03a6e1649c7891860efa07c82bf00d6c28272047d89a16a081b3d33/app_model-0.2.2-py3-none-any.whl.metadata
  Using cached app_model-0.2.2-py3-none-any.whl.metadata (3.3 kB)
  Using cached cachey-0.2.1-py3-none-any.whl (6.4 kB)
  Obtaining dependency information for certifi>=2018.1.18 from http

In [7]:
from dw3d import geometry_reconstruction_3d
from dw3d.mask_reconstruction import reconstruct_mask_from_dict


In [1]:
import skimage.io as io
from dw3d import GeometryReconstruction3D
from dw3d.mask_reconstruction import reconstruct_mask_from_dict


def make_idx(n):
    return(("000"+str(n))[-3:])

dist = 3

image_path = "../data/Images/1.tif"
image = io.imread(image_path)
DW = GeometryReconstruction3D(image,min_dist = dist, expansion_labels =0,print_info=True)
DW.export_mesh("actual_2.rec")

Mode == Skimage
min_distance = 3
Computing EDT ...
EDT computed in  0.53
Searching local extremas ...
Number of local minimas : 4804
Number of local maxes : 153
Local minimas computed in  0.37
Starting triangulation..
Triangulation build in  0.29
Graph build in  0.573


IndexError: arrays used as indices must be of integer (or boolean) type

## 2) Reconstruct cell segmentation masks. Visualize in napari

In [2]:
# must install Napari first (and polyscope if polyscope function is used)
# from dw3d.viewing import plot_cells_polyscope
#plot_cells_polyscope(DW)
from dw3d.viewing import plot_in_napari
v = plot_in_napari(DW, add_mesh=True)

/home/perez/Documents/Sacha/delaunay_watershed_3D/.venv/lib/python3.11/site-packages/napari/layers/utils/color_transformations.py:53: UserWarning: The provided colors parameter has 2 entries, while the data contains 3 entries. Setting colors to white.
  warnings.warn(


In [5]:
DW.return_mesh()

(array([[  0.,   0.,   0.],
        [  0.,   0., 156.],
        [  0., 199.,   0.],
        ...,
        [142., 120.,  82.],
        [142., 122.,  75.],
        [143., 118.,  78.]]),
 array([[ 101,  102,  107,    0,    1],
        [ 101,  111,  102,    0,    1],
        [ 101,  104,  106,    0,    1],
        ...,
        [2710, 2715, 2712,    0,    1],
        [2711, 2713, 2715,    0,    1],
        [2712, 2715, 2714,    0,    1]]))

In [ ]:
DW.export_mesh("actual_2.rec")

In [6]:
DW.export_segmentation("Dict_mask.npy")

In [7]:
labels = reconstruct_mask_from_dict("Dict_mask.npy")

In [8]:
import napari

v = napari.view_image(labels)
v.add_image(DW.labels)

<Image layer 'Image' at 0x7efbe488f250>

## 3) Compare files sizes: mesh VS masks 

In [9]:
import numpy as np

D = np.load("Dict_mask.npy",allow_pickle=True).item()
Dict_memory_footprint = D["Verts"].nbytes + D["Faces"].nbytes + D["seeds"].nbytes + D["image_shape"].nbytes
print("Size of Delaunay-Watershed data:",Dict_memory_footprint,"bytes")
print("Size of the labels:",labels.nbytes,"bytes")
print("Relative size:", Dict_memory_footprint/labels.nbytes)

Size of Delaunay-Watershed data: 274400 bytes
Size of the labels: 39438400 bytes
Relative size: 0.006957685910178912
